# HGNN代码复现

* [源仓库地址](https://github.com/iMoonLab/HGNN)

### 导入相关python库和文件

In [1]:
import os
import time
import copy
import torch
import torch.optim as optim
import pprint as pp
import utils.hypergraph_utils as hgut
from models import HGNN
from config import get_config
from datasets import load_feature_construct_H

### 导入模型的设置

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cfg = get_config('config/config.yaml')

making direction ./result/hgnn!
making direction ./result/hgnn\ckpt!
making direction ./result/hgnn\hypergraph_ModelNet40!


In [4]:
cfg

{'data_root': './data/features',
 'modelnet40_ft': './data/features\\ModelNet40_mvcnn_gvcnn.mat',
 'ntu2012_ft': './data/features\\NTU2012_mvcnn_gvcnn.mat',
 'graph_type': 'hypergraph',
 'K_neigs': [10],
 'm_prob': 1.0,
 'is_probH': True,
 'use_mvcnn_feature_for_structure': True,
 'use_gvcnn_feature_for_structure': True,
 'on_dataset': 'ModelNet40',
 'use_mvcnn_feature': False,
 'use_gvcnn_feature': True,
 'result_root': './result/hgnn',
 'result_sub_folder': './result/hgnn\\hypergraph_ModelNet40',
 'ckpt_folder': './result/hgnn\\ckpt',
 'max_epoch': 600,
 'n_hid': 128,
 'lr': 0.001,
 'milestones': [100],
 'gamma': 0.9,
 'drop_out': 0.5,
 'print_freq': 50,
 'weight_decay': 0.0005,
 'decay_step': 200,
 'decay_rate': 0.7}

### 数据集路径

In [5]:
# initialize data
data_dir = cfg['modelnet40_ft'] if cfg['on_dataset'] == 'ModelNet40' \
    else cfg['ntu2012_ft']

In [6]:
data_dir

'./data/features\\ModelNet40_mvcnn_gvcnn.mat'

In [7]:
# gvcnn_ft特征; label; 训练集索引; 测试集; 两个数据的最近n个邻居矩阵
fts, lbls, idx_train, idx_test, H = \
    load_feature_construct_H(data_dir,
                             m_prob=cfg['m_prob'],
                             K_neigs=cfg['K_neigs'],
                             is_probH=cfg['is_probH'],
                             use_mvcnn_feature=cfg['use_mvcnn_feature'],
                             use_gvcnn_feature=cfg['use_gvcnn_feature'],
                             use_mvcnn_feature_for_structure=cfg['use_mvcnn_feature_for_structure'],
                             use_gvcnn_feature_for_structure=cfg['use_gvcnn_feature_for_structure'])

Constructing hypergraph incidence matrix! 
(It may take several minutes! Please wait patiently!)


In [8]:
G = hgut.generate_G_from_H(H)  # H-超边矩阵

In [9]:
n_class = int(lbls.max()) + 1  # 类别数量;
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [10]:
n_class

40

In [11]:
# transform data to device
fts = torch.Tensor(fts).to(device)
lbls = torch.Tensor(lbls).squeeze().long().to(device)
G = torch.Tensor(G).to(device)
idx_train = torch.Tensor(idx_train).long().to(device)
idx_test = torch.Tensor(idx_test).long().to(device)

In [12]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, print_freq=500):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())  # 复制参数
    best_acc = 0.0

    for epoch in range(num_epochs):
        if epoch % print_freq == 0:
            print('-' * 10)
            print(f'Epoch {epoch}/{num_epochs - 1}')

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            idx = idx_train if phase == 'train' else idx_test

            # Iterate over data.
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(fts, G)  # [12311, 2048]; G:超图矩阵
                loss = criterion(outputs[idx], lbls[idx])
                _, preds = torch.max(outputs, 1)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * fts.size(0)
            running_corrects += torch.sum(preds[idx] == lbls.data[idx])

            epoch_loss = running_loss / len(idx)
            epoch_acc = running_corrects.double() / len(idx)

            if epoch % print_freq == 0:
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        if epoch % print_freq == 0:
            print(f'Best val Acc: {best_acc:4f}')
            print('-' * 20)

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [13]:
print(f"Classification on {cfg['on_dataset']} dataset!!! class number: {n_class}")
print(f"use MVCNN feature: {cfg['use_mvcnn_feature']}")
print(f"use GVCNN feature: {cfg['use_gvcnn_feature']}")
print(f"use MVCNN feature for structure: {cfg['use_mvcnn_feature_for_structure']}")
print(f"use GVCNN feature for structure: {cfg['use_gvcnn_feature_for_structure']}")
print('Configuration -> Start')
pp.pprint(cfg)
print('Configuration -> End')

Classification on ModelNet40 dataset!!! class number: 40
use MVCNN feature: False
use GVCNN feature: True
use MVCNN feature for structure: True
use GVCNN feature for structure: True
Configuration -> Start
{'K_neigs': [10],
 'ckpt_folder': './result/hgnn\\ckpt',
 'data_root': './data/features',
 'decay_rate': 0.7,
 'decay_step': 200,
 'drop_out': 0.5,
 'gamma': 0.9,
 'graph_type': 'hypergraph',
 'is_probH': True,
 'lr': 0.001,
 'm_prob': 1.0,
 'max_epoch': 600,
 'milestones': [100],
 'modelnet40_ft': './data/features\\ModelNet40_mvcnn_gvcnn.mat',
 'n_hid': 128,
 'ntu2012_ft': './data/features\\NTU2012_mvcnn_gvcnn.mat',
 'on_dataset': 'ModelNet40',
 'print_freq': 50,
 'result_root': './result/hgnn',
 'result_sub_folder': './result/hgnn\\hypergraph_ModelNet40',
 'use_gvcnn_feature': True,
 'use_gvcnn_feature_for_structure': True,
 'use_mvcnn_feature': False,
 'use_mvcnn_feature_for_structure': True,
 'weight_decay': 0.0005}
Configuration -> End


In [14]:
model_ft = HGNN(in_ch=fts.shape[1],  # 原始特征维度
                n_class=n_class,  # 类别数量
                n_hid=cfg['n_hid'],  # 隐层特征
                dropout=cfg['drop_out'])  # dropout
model_ft = model_ft.to(device)

optimizer = optim.Adam(model_ft.parameters(), lr=cfg['lr'],
                        weight_decay=cfg['weight_decay'])

# optimizer = optim.SGD(model_ft.parameters(), lr=0.01, weight_decay=cfg['weight_decay)
schedular = optim.lr_scheduler.MultiStepLR(optimizer,
                                            milestones=cfg['milestones'],
                                            gamma=cfg['gamma'])
criterion = torch.nn.CrossEntropyLoss()  # loss函数


In [20]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
model_ft = train_model(model_ft, criterion, optimizer, schedular, num_epochs=5, print_freq=cfg['print_freq']) # cfg['max_epoch']

----------
Epoch 0/4
train Loss: 0.6848 Acc: 0.9343
val Loss: 2.9660 Acc: 0.9327
Best val Acc: 0.932739
--------------------

Training complete in 0m 3s
Best val Acc: 0.950972
